use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle


/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_2_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [3]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [4]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [5]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [6]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [7]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge valid score

In [8]:
FIELD_NAME = 'sasrec_feat_scores'

In [9]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_DE/product_embeddings_2023-05-22-14-49-09.pt'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_DE/valid_embeddings_2023-05-22-14-50-10.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_JP/product_embeddings_2023-05-22-14-56-18.pt'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_JP/valid_embeddings_2023-05-22-14-56-53.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_UK/product_embeddings_2023-05-22-15-00-18.pt'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_UK/valid_embeddings_2023-05-22-15-01-04.pt'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/c76eddf0a07106ffcce7ce8010856a3b'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/81a71d0a18766af84b3beab69bf53e69'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/250dbc09c30162452e00486051e47756'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [13]:
# sess embeddings 
valid_DE_query_emb = torch.load(DE_valid_embeddings_path, map_location='cpu')
valid_JP_query_emb = torch.load(JP_valid_embeddings_path, map_location='cpu')
valid_UK_query_emb = torch.load(UK_valid_embeddings_path, map_location='cpu')
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [14]:
# product_embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [15]:
merged_candidates_ = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [16]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [17]:
# merged_candidates_g = cudf.from_pandas(merged_candidates_)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
# merged_candidates_score_g = merged_candidates_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_score_g['dataset_id'] = merged_candidates_score_g['dataset_id'].fillna(0)
# merged_candidates_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_score_g = merged_candidates_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_score = merged_candidates_score_g.to_pandas()

In [ ]:
merged_candidates_score = merged_candidates_.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_score['dataset_id'] = merged_candidates_score['dataset_id'].fillna(0)
merged_candidates_score.drop(columns=['locale'], inplace=True)
merged_candidates_score = merged_candidates_score.sort_values(by=['sess_id', 'product'])
merged_candidates_score.reset_index(drop=True, inplace=True)

In [19]:
# del merged_candidates_g
# del product_id_df_g
# del merged_candidates_score_g

In [20]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \
        merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_839857/2978421209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \


In [21]:
valid_query_embeddings = valid_query_embeddings.to('cuda:0')
product_embeddings = product_embeddings.to('cuda:0')

In [22]:
merged_candidates_score[FIELD_NAME] = get_scores(merged_candidates_score, valid_query_embeddings, product_embeddings)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 41215/41215 [00:43<00:00, 950.07it/s]


In [23]:
normalize_scores(merged_candidates_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [24]:
merged_candidates[FIELD_NAME] = merged_candidates_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_score['normalized_'+FIELD_NAME]

In [25]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [26]:
merged_candidates

,sess_id,sess_locale,product,target,sasrec_scores_2,sasrec_normalized_scores_2,product_freq,gru4rec_scores,gru4rec_normalized_scores,sess_avg_price,...,sasrec_normalized_scores_3,normalized_all_items_co_graph_count_0,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,narm_scores,normalized_narm_scores,sasrec_feat_scores,normalized_sasrec_feat_scores
0,0,UK,B000OPPVCS,0.0,11.972421,2.286162e-04,104,6.484859,3.816029e-05,7.388571,...,2.517129e-04,0.002635,2,8.378721,1.007163e-05,362.381012,16.883379,2.761682e-04,15.763786,3.403873e-03
1,0,UK,B000V599Y2,0.0,13.152878,7.443427e-04,37,4.342063,4.477209e-06,7.388571,...,2.031618e-04,0.003953,3,7.534612,4.330198e-06,64.592583,13.755573,1.209999e-05,11.217986,3.612080e-05
2,0,UK,B0018HH444,0.0,5.606023,3.928400e-07,7,3.220763,1.458925e-06,7.388571,...,2.036883e-06,0.001318,1,11.199949,1.691779e-04,67.996689,9.759952,2.225914e-07,6.324236,2.706629e-07
3,0,UK,B0079JI4DU,0.0,0.000000,1.443945e-09,67,0.000000,5.824698e-08,7.388571,...,4.685961e-09,0.002635,2,0.000000,2.313489e-09,0.000000,0.000000,1.284740e-11,0.000000,4.851186e-10
4,0,UK,B0079JI4EY,0.0,0.000000,1.443945e-09,77,0.000000,5.824698e-08,7.388571,...,4.685961e-09,0.002635,2,0.000000,2.313489e-09,0.000000,0.000000,1.284740e-11,0.000000,4.851186e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84407334,361580,DE,B0BB7XV97M,0.0,9.117821,6.077226e-05,56,9.268379,1.396883e-05,32.424000,...,3.403967e-05,0.003356,2,13.965775,2.930834e-03,0.000000,9.397533,2.219567e-04,8.051070,1.486921e-06
84407335,361580,DE,B0BB7YSRBX,0.0,9.163816,6.363281e-05,58,7.047796,1.516259e-06,32.424000,...,2.115080e-05,0.001678,1,10.212118,6.867505e-05,0.000000,8.080528,5.947030e-05,8.144089,1.631869e-06
84407336,361580,DE,B0BB7ZMGY8,0.0,11.256460,5.158278e-04,452,9.359167,1.529639e-05,32.424000,...,5.522656e-05,0.038591,23,12.275789,5.408041e-04,0.000000,10.893625,9.908607e-04,11.075905,3.061659e-05
84407337,361580,DE,B0BD4CP7N3,0.0,-3.778687,1.523433e-10,1,-0.593306,7.282568e-10,32.424000,...,4.433373e-10,0.000000,0,-2.456360,2.162463e-10,36.028561,2.628504,2.549859e-07,1.995993,3.488199e-09


In [27]:
# verify scores
merged_candidates[merged_candidates['sess_id'] == 150001].sort_values(by=['sasrec_scores_2'], ascending=False)[['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_sasrec_feat_scores', 'sasrec_feat_scores']].iloc[:15]

,sess_locale,product,sasrec_normalized_scores_3,sasrec_scores_3,normalized_sasrec_feat_scores,sasrec_feat_scores
35011902,DE,B08VN4VTMC,0.407627,18.442009,0.441094,19.765358
35011843,DE,B07TP1HY5B,0.173127,17.585684,0.134083,18.574560
35011977,DE,B0BBRKH55W,0.131551,17.311052,0.114731,18.418694
35011978,DE,B0BBRPFP73,0.095298,16.988661,0.134298,18.576164
35011979,DE,B0BBRR84KK,0.074192,16.738312,0.084972,18.118423
35011888,DE,B08HHXDHRJ,0.047119,16.284338,0.015333,16.406120
35011807,DE,B0793LCHRD,0.015908,15.198477,0.009762,15.954628
35011786,DE,B011KJ6WLU,0.010079,14.742159,0.022817,16.803595
35011790,DE,B01MSZ7WK7,0.009994,14.733601,0.000751,13.389692
35011968,DE,B0B7NCTHWX,0.009510,14.684032,0.020397,16.691465
